In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
os.chdir('..')
sys.path.append('src')

In [ ]:
import torch
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
plt.figure(figsize=(10, 10))

from postprocessing import SmoothTiles
from utils import create_dir, read_frame, save_tiff_uint8_single_band
from postprocessing import postprocess, plot_img

# General

In [ ]:
def test_inf(batch):
    return batch[:, :1]

img = postprocess(test_inf, '/mnt/storage/HuBMAP/train/', './output', True)

In [ ]:
plot_img(img)

# Generate Test

In [ ]:
def get_random_prediction(size=128, n=2, cuda=True):
    device = 'cuda' if cuda and torch.cuda.is_available() else 'cpu'
    img_main = torch.normal(0, 1, size=(size, size, n**2)).to(device)
    img_sub = torch.normal(0, 1, size=(size, size, (n+1)**2)).to(device)
    return img_main, img_sub

def get_black_prediction(size=128, n=2, cuda=True):
    device = 'cuda' if cuda and torch.cuda.is_available() else 'cpu'
    img_main = torch.zeros((size, size, n**2)).to(device)
    img_sub = torch.ones((size, size, (n+1)**2)).to(device)
    return img_main, img_sub

# img_main, img_sub = get_random_prediction(size_w, 3)
img_main, img_sub = get_black_prediction(size_w, 3)
print(img_main.shape, img_sub.shape)

# Show window

In [ ]:
st = SmoothTiles()

In [ ]:
plt.plot(st.triangle(size_w))

In [ ]:
plt.imshow(st.window_2D(size_w, st.triangle))

In [ ]:
plt.plot(st.gauss(size_w, 20))

In [ ]:
plt.imshow(st.window_2D(size_w, st.gauss))

In [ ]:
plt.plot(st.spline(size_w))

In [ ]:
plt.imshow(st.window_2D(size_w, st.spline))

# Smooth Tiles

In [ ]:
def concat(img):
    n = int(np.sqrt(img.shape[-1]))
    big_img = torch.reshape(img, (img.shape[0], img.shape[1], n, n))
    sub_img = []
    for i in range(n):
        tpl_img = tuple([j for j in big_img[:, :, :, i].permute(2, 0, 1)])
        sub_img.append(np.concatenate(tpl_img, axis=1))
    return np.concatenate(tuple(sub_img), axis=0)

In [ ]:
smooth = st.merge(img_main, img_sub)
print(smooth.shape)
plt.imshow(concat(smooth.to('cpu')))